In [70]:
import plotly.graph_objects as go
# x,y,z 좌표를 가지고 있는 두 데이터프레임을 입력받아 3D 그래프를 생성하는 함수
def points_on_3d(df_coord_1, df_coord_2):
    # 3D 그래프 생성
    fig = go.Figure()

    # df 데이터프레임의 좌표를 빨간색으로 표시
    fig.add_trace(go.Scatter3d(
        x=df_coord_1['x'],
        y=df_coord_1['y'],
        z=df_coord_1['z'],
        mode='markers',
        marker=dict(
            size=1,
            color='red',
            opacity=0.8
        )
    ))

    # df_coordinates_filtered 데이터프레임의 좌표를 파란색으로 표시
    fig.add_trace(go.Scatter3d(
        x=df_coord_2['x'],
        y=df_coord_2['y'],
        z=df_coord_2['z'],
        mode='markers',
        marker=dict(
            size=1,
            color='blue',
            opacity=0.5
        )
    ))

    # 그래프 표시
    fig.show()

In [90]:
import pandas as pd
from Bio.PDB import PDBParser
import pandas as pd

# PDB 파일 읽기
parser = PDBParser()
structure = parser.get_structure("1crn", "1CRN.pdb")

# 원자 좌표를 저장할 데이터테이블 초기화
data = {'atom': [], 'x': [], 'y': [], 'z': []}

# 모든 원자를 순회하며 좌표를 데이터테이블에 추가
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                data['atom'].append(atom.get_name())
                data['x'].append(atom.get_coord()[0])
                data['y'].append(atom.get_coord()[1])
                data['z'].append(atom.get_coord()[2])

# 데이터테이블 생성
df_protein = pd.DataFrame(data)

print(df_protein)

    atom          x       y       z
0      N  17.047001  14.099   3.625
1     CA  16.966999  12.784   4.338
2      C  15.685000  12.755   5.133
3      O  15.268000  13.825   5.594
4     CB  18.170000  12.703   5.337
..   ...        ...     ...     ...
322   CB  12.266000   4.769  13.501
323   CG  12.538000   4.304  14.922
324  OD1  11.982000   4.849  15.886
325  ND2  13.407000   3.298  15.015
326  OXT  12.703000   4.973  10.746

[327 rows x 4 columns]


In [92]:
import numpy as np

# 데이터프레임의 x, y, z 좌표의 최소값과 최대값을 계산
min_x = df_protein['x'].min()
max_x = df_protein['x'].max()

min_y = df_protein['y'].min()
max_y = df_protein['y'].max()

min_z = df_protein['z'].min()
max_z = df_protein['z'].max()

# x, y, z 축의 범위 계산
x_range = np.arange(min_x - 5, max_x + 5, 5)
y_range = np.arange(min_y - 5, max_y + 5, 5)
z_range = np.arange(min_z - 5, max_z + 5, 5)

# 모든 좌표 생성
coordinates = np.array(np.meshgrid(x_range, y_range, z_range)).T.reshape(-1,3)

# 데이터프레임으로 변환
df_coordinates = pd.DataFrame(coordinates, columns=['x', 'y', 'z'])

print(len(df_protein))
print(len(df_coordinates))
points_on_3d(df_protein, df_coordinates)


327
448


In [93]:
from scipy.spatial import distance

# df_proteins에 있는 각 좌표와 df_coordinates에 있는 각 좌표 사이의 거리를 계산
distances = distance.cdist(df_coordinates[['x', 'y', 'z']].values, df_protein[['x', 'y', 'z']].values, 'euclidean')

# 거리가 5 이하 3 초과인 좌표를 선택
df_coordinates_filtered = df_coordinates[np.logical_and(np.min(distances, axis=1) <= 5, np.min(distances, axis=1) > 3)]

print(len(df_coordinates_filtered))
points_on_3d(df_protein, df_coordinates_filtered)

43


In [94]:
# KMeans 클러스터링으로 좌표를 1/2로 줄임

from sklearn.cluster import KMeans

cluster_num = int(len(df_coordinates_filtered)/2)

# KMeans 객체 생성
kmeans = KMeans(n_clusters=cluster_num, random_state=0)

# 클러스터링 수행
kmeans.fit(df_coordinates_filtered)

# 클러스터 중심점을 데이터프레임으로 변환
df_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['x', 'y', 'z'])

print(len(df_centroids))
points_on_3d(df_protein, df_centroids)

21


/home/siu/anaconda/envs/chem/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [85]:
# 생성된 좌표에 Br를 만들어서 PDB 파일로 저장

from Bio.PDB import PDBIO, Atom, Residue, Chain, Model, Structure

# 새로운 PDB 구조 생성
structure = Structure.Structure('new_structure')

# 모델 생성
model = Model.Model(0)
structure.add(model)

# 체인 생성
chain = Chain.Chain('A')
model.add(chain)

# df_centroids에 있는 각 좌표에 대해 브롬 원자를 생성
for i, row in df_centroids.iterrows():
    # 브롬 원자 생성
    atom = Atom.Atom('BR', row[['x', 'y', 'z']].values, 0.0, 1.0, ' ', 'BR', 1, 'BR')
    
    # 잔기 생성
    residue = Residue.Residue((' ', i, ' '), 'BR', ' ')
    residue.add(atom)
    
    # 체인에 잔기 추가
    chain.add(residue)

# PDB 파일로 저장
io = PDBIO()
io.set_structure(structure)
io.save('generated_coordinates_Br.pdb')